In [238]:
# Import dependencies
import pandas as pd
import os
from dotenv import load_dotenv    # from Karen's code
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy import select      # Not used
import psycopg2
import csv   # Not used
import numpy as np   # Not used
from pprint import pprint
import json


# import gzip    # Not used

In [239]:

url_metar_gz="https://aviationweather.gov/data/cache/metars.cache.csv.gz"
url_TAF_gz="https://aviationweather.gov/data/cache/tafs.cache.csv.gz"
url_airsigmets_gz="https://aviationweather.gov/data/cache/airsigmets.cache.csv.gz"

__To retrieve METAR data__

In [240]:
# Fetch, load, and decompress the data relative to metar
metar_data_df = pd.read_csv(url_metar_gz, header=5, compression='gzip')
metar_data_df.head()

,raw_text,station_id,observation_time,latitude,longitude,temp_c,dewpoint_c,wind_dir_degrees,wind_speed_kt,wind_gust_kt,...,maxT24hr_c,minT24hr_c,precip_in,pcp3hr_in,pcp6hr_in,pcp24hr_in,snow_in,vert_vis_ft,metar_type,elevation_m
0,CYHU 291558Z AUTO 02004KT 1 3/4SM -RA BR OVC00...,CYHU,2023-12-29T15:58:00Z,45.516,-73.415,2.0,1.0,20,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPECI,26.0
1,PHNG 291557Z 09009KT 7SM FEW034 BKN043 BKN060 ...,PHNG,2023-12-29T15:57:00Z,21.450,-157.763,24.4,18.3,90,9.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,METAR,1.0
2,KMNM 291557Z AUTO 35004KT 10SM CLR 01/M02 A298...,KMNM,2023-12-29T15:57:00Z,45.126,-87.639,0.6,-2.2,350,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,METAR,188.0
3,CYRQ 291557Z AUTO 05010KT 2 1/2SM -RA -UP BR O...,CYRQ,2023-12-29T15:57:00Z,46.358,-72.680,1.0,1.0,50,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPECI,58.0
4,CYPX 291557Z AUTO 06007KT 1/8SM +SN SCT006 OVC...,CYPX,2023-12-29T15:57:00Z,60.045,-77.281,-14.0,-16.0,60,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPECI,10.0


In [241]:
# Convert the dataframe to a JSON format
# metar_json = json.loads(json.dumps(list(metar_data_df.T.to_dict().values())))   # https://stackoverflow.com/questions/39257147/convert-pandas-dataframe-to-json-format answer #10 Amir.S
# pprint(metar_json, sort_dicts=False)  # the pprint process is slow (10.5sec)

In [242]:
# pprint(metar_json, sort_dicts=False)

In [243]:
# Convert some columns to INT
# cols=['wind_dir_degrees','wind_speed_kt','wind_gust_kt','cloud_base_ft_agl','cloud_base_ft_agl.2',\
#     'cloud_base_ft_agl.3','vert_vis_ft','elevation_m']
# metar_data_df[cols]=metar_data_df[cols].apply(pd.to_numeric, errors='coerce', downcast='integer', axis=1) # Does not appear to convert to INT, but to FLOAT64
# metar_data_df[cols]=metar_data_df[cols].astype(int)   # Cannot convert NaN

In [244]:
# metar_data_df.columns

In [245]:
# metar_data_df.info()

In [246]:
# convert the dataframe to a dictionary then to a JSON string
metar_string=json.dumps(list(metar_data_df.T.to_dict().values()))

# open the file in write mode
# output_path = os.path.join("Resources", "metar_data.json")  # To be removed if logic.js cannot read from Resources
output_path = os.path.join("", "metar_data.json")
with open(output_path, "w") as file:
    # write the JSON string to the file
    file.write(metar_string.replace("NaN","null"))

# file is automatically closed after the with block

In [247]:
# To save the dataframe as a csv file for future import to database (not needed anymore)
# output_path2 = os.path.join("", "metar_data.csv")
# metar_data_df.to_csv(output_path2, index=False)

# file is automatically closed after the with block

__To refresh the metar table in the Render database__

In [248]:
# Test to refresh the metar table

load_dotenv()
db_url = os.environ.get("link_render")
connection = psycopg2.connect(db_url)
cursor = connection.cursor()
cursor.execute("DELETE FROM metar;")    # Deletes all the rows but keep the table
connection.commit()
cursor.close()
connection.close()

In [249]:
# Test to populate the empty table from a json file
load_dotenv()
db_url = os.environ.get("link_render")
connection = psycopg2.connect(db_url)
cursor = connection.cursor()

cursor.execute("set search_path to public") # https://dba.stackexchange.com/questions/268365/using-python-to-insert-json-into-postgresql

with open('metar_data.json') as file:
    data = file.read()

query_sql = """
INSERT INTO metar SELECT * FROM
json_populate_recordset(NULL::metar, %s);
"""

cursor.execute(query_sql, (data,))
connection.commit()

__To download a new csv file made of the joining of airport data and weather data__

In [250]:
# download the output of a query across multiple tables with all the all the active public airport with or without weather information.
# Will be used to populate the makers that do not have METAR information.
# There is only one row per airport so only one runway is listed even if the airport as more.
# More parameters can be added to complete the info on the popup window.

load_dotenv()
db_url = os.environ.get("link_render")

# query="""
#     SELECT DISTINCT ON (arpt_id)
# 	arpt_id, icao_id, metar.station_id, arpt_name, apt_rwy.rwy_id, lat_decimal, long_decimal, metar.observation_time, metar.wind_speed_kt, metar.flight_category, metar.raw_text
#     FROM apt_rwy
#     JOIN apt_base ON apt_base.site_no = apt_rwy.site_no
#     FULL JOIN metar ON metar.station_id = apt_base.icao_id
#     WHERE facility_use_code='PU' AND site_type_code='A' AND arpt_status='O';
#     """

query="""
    SELECT DISTINCT ON (arpt_id)
    arpt_id, icao_id, metar.station_id, arpt_name, apt_rwy.rwy_id, lat_decimal, long_decimal, metar.observation_time, metar.wind_speed_kt, metar.flight_category, metar.raw_text, elev, metar.visibility_statute_mi, metar.cloud_base_ft_agl
    FROM apt_rwy
    JOIN apt_base ON apt_base.site_no = apt_rwy.site_no
    FULL JOIN metar ON (RIGHT(metar.station_id, LENGTH(metar.station_id) - 1)) = apt_base.arpt_id
    WHERE facility_use_code='PU' AND site_type_code='A' AND arpt_status='O' AND (@(lat_decimal - metar.latitude) <1) AND (@(long_decimal - metar.longitude) <1)
    """

engine=create_engine(db_url)
with engine.begin() as conn:
    results=conn.execute(
        text(query)
    )

arpt_weather_query_df = pd.DataFrame(results)

# Save the df as a CSV file. Might not be needed if we only use JSON    TO BE REMOVED?
# arpt_weather_query_df.to_csv (r'airport_weather_data.csv', index = False) # place 'r' before the path name


# convert the dataframe to a dictionary then to a JSON string
arpt_weather_string=json.dumps(list(arpt_weather_query_df.T.to_dict().values()))

# open the file in write mode
# output_path = os.path.join("Resources", "metar_data.json")  # To be removed if logic.js cannot read from Resources
output_path = os.path.join("", "airport_weather_data.json")
with open(output_path, "w") as file:
    # write the JSON string to the file
    file.write(arpt_weather_string.replace("NaN","null"))

# file is automatically closed after the with block



In [251]:
# arpt_weather_query_df.head()

In [252]:
# pprint(arpt_weather_string)

__To retrieve TAF data__

In [253]:
# Fetch, load, and decompress the data relative to TAF  (working but not used for now)
# taf_data_df = pd.read_csv(url_TAF_gz, header=5,index_col=False, compression='gzip',dtype='str',low_memory=False)
# taf_data_df.head()

In [254]:
# Convert the dataframe to a JSON format  (working but not used for now)
# taf_json = json.loads(json.dumps(list(taf_data_df.T.to_dict().values())))   # https://stackoverflow.com/questions/39257147/convert-pandas-dataframe-to-json-format answer #10 Amir.S
# pprint(taf_json, sort_dicts=False)  # the pprint process is slow (10.5sec)

In [255]:
# # convert the dataframe to a dictionary then to a JSON string  (working but not used for now)
# taf_string=json.dumps(list(taf_data_df.T.to_dict().values()))

# # open the file in write mode
# output_path = os.path.join("Resources", "taf_data.json")
# with open(output_path, "w") as file:
#     # write the JSON string to the file
#     file.write(taf_string.replace("NaN","null"))

# # file is automatically closed after the with block

__To retrieve AIRMET and SIGMET polygons data__

In [256]:
# Fetch, load, and decompress the data relative to Airmet and Sigmet
airsigmet_data_df = pd.read_csv(url_airsigmets_gz, header=5, compression='gzip', encoding='utf-8')
airsigmet_data_df.head()

,raw_text,valid_time_from,valid_time_to,lon:lat points,min_ft_msl,max_ft_msl,movement_dir_degrees,movement_speed_kt,hazard,severity,airsigmet_type
0,WSUS32 KKCI 291555 SIGC CONVECTIVE SIGMET.....,2023-12-29T15:55:00Z,2023-12-29T17:55:00Z,-107:49;-107:31.79;-106.52:31.78;-106.48:31.75...,NaN,NaN,NaN,NaN,CONVECTIVE,1,SIGMET
1,WSUS31 KKCI 291555 SIGE CONVECTIVE SIGMET.....,2023-12-29T15:55:00Z,2023-12-29T17:55:00Z,-87:27.48;-84.26:27.48;-83.01:24;-78.66:24;-79...,NaN,NaN,NaN,NaN,CONVECTIVE,1,SIGMET
2,WSUS33 KKCI 291555 SIGW CONVECTIVE SIGMET.....,2023-12-29T15:55:00Z,2023-12-29T17:55:00Z,-122.77:49;-123.12:48.96;-123.02:48.6;-123.16:...,NaN,NaN,NaN,NaN,CONVECTIVE,1,SIGMET
3,WAUS41 KKCI 291445 BOSS WA 291445 AIRMET SIE...,2023-12-29T14:45:00Z,2023-12-29T21:00:00Z,-69.3144:47.5893;-67.8065:47.2344;-66.7115:44....,0.0,1000.0,NaN,NaN,IFR,0,AIRMET
4,WAUS41 KKCI 291445 BOSS WA 291445 AIRMET SIE...,2023-12-29T14:45:00Z,2023-12-29T21:00:00Z,-77.2526:42.8355;-75.1583:41.2609;-77.02:40.23...,NaN,NaN,NaN,NaN,MTN OBSCN,0,AIRMET


In [257]:
# replace with <br> in raw_text
# airsigmet_data_df['raw_text'].str.replace(r' \x07','<br>')
# airsigmet_data_df.replace(r' x07','<br>', regex=True)

# To convert the points delimiting the areas into something Leaflet-friendly
for j in range(len(airsigmet_data_df)):
    test=airsigmet_data_df.iloc[j]['lon:lat points']
    if  pd.isna(test)!=True:    # Test if the cell is not NaN
        test1=test.split(';')
        for i in range(len(test1)):
            test1[i]=test1[i].split(':')    # Creates list of coordinates
            test1[i][0],test1[i][1]=float(test1[i][1]),float(test1[i][0])   # Swap lon:lat to lat:lon
        airsigmet_data_df.at[j,'lon:lat points']=test1

    # shift cells for missing column
    test2=airsigmet_data_df.iloc[j]['airsigmet_type']
    if  pd.isna(test2)==True:    # Test if the cell is NaN
        airsigmet_data_df.at[j,'airsigmet_type']=airsigmet_data_df.at[j,'severity']
        airsigmet_data_df.at[j,'severity']=airsigmet_data_df.at[j,'hazard']
        airsigmet_data_df.at[j,'hazard']=airsigmet_data_df.at[j,'movement_speed_kt']
        airsigmet_data_df.at[j,'movement_speed_kt']=airsigmet_data_df.at[j,'movement_dir_degrees']
        airsigmet_data_df.at[j,'movement_dir_degrees']=airsigmet_data_df.at[j,'max_ft_msl']
        airsigmet_data_df.at[j,'max_ft_msl']=airsigmet_data_df.at[j,'min_ft_msl']
        airsigmet_data_df.at[j,'min_ft_msl']="NaN"

  


airsigmet_data_df.rename(columns={"lon:lat points":"lat_lon_points"}, inplace=True) # Update the column name
airsigmet_data_df.head()

,raw_text,valid_time_from,valid_time_to,lat_lon_points,min_ft_msl,max_ft_msl,movement_dir_degrees,movement_speed_kt,hazard,severity,airsigmet_type
0,WSUS32 KKCI 291555 SIGC CONVECTIVE SIGMET.....,2023-12-29T15:55:00Z,2023-12-29T17:55:00Z,"[[49.0, -107.0], [31.79, -107.0], [31.78, -106...",NaN,NaN,NaN,NaN,CONVECTIVE,1,SIGMET
1,WSUS31 KKCI 291555 SIGE CONVECTIVE SIGMET.....,2023-12-29T15:55:00Z,2023-12-29T17:55:00Z,"[[27.48, -87.0], [27.48, -84.26], [24.0, -83.0...",NaN,NaN,NaN,NaN,CONVECTIVE,1,SIGMET
2,WSUS33 KKCI 291555 SIGW CONVECTIVE SIGMET.....,2023-12-29T15:55:00Z,2023-12-29T17:55:00Z,"[[49.0, -122.77], [48.96, -123.12], [48.6, -12...",NaN,NaN,NaN,NaN,CONVECTIVE,1,SIGMET
3,WAUS41 KKCI 291445 BOSS WA 291445 AIRMET SIE...,2023-12-29T14:45:00Z,2023-12-29T21:00:00Z,"[[47.5893, -69.3144], [47.2344, -67.8065], [44...",0.0,1000.0,NaN,NaN,IFR,0,AIRMET
4,WAUS41 KKCI 291445 BOSS WA 291445 AIRMET SIE...,2023-12-29T14:45:00Z,2023-12-29T21:00:00Z,"[[42.8355, -77.2526], [41.2609, -75.1583], [40...",NaN,NaN,NaN,NaN,MTN OBSCN,0,AIRMET


In [258]:
# Convert the dataframe to a JSON format
airsigmet_json = json.loads(json.dumps(list(airsigmet_data_df.T.to_dict().values())))   # https://stackoverflow.com/questions/39257147/convert-pandas-dataframe-to-json-format answer #10 Amir.S
pprint(airsigmet_json, sort_dicts=False)

[{'raw_text': 'WSUS32 KKCI 291555 \x07SIGC  \x07CONVECTIVE SIGMET...NONE \x07 '
              '\x07OUTLOOK VALID 291755-292155 \x07TS ARE NOT EXPD.',
  'valid_time_from': '2023-12-29T15:55:00Z',
  'valid_time_to': '2023-12-29T17:55:00Z',
  'lat_lon_points': [[49.0, -107.0],
                     [31.79, -107.0],
                     [31.78, -106.52],
                     [31.75, -106.48],
                     [31.75, -106.38],
                     [31.48, -106.22],
                     [31.4, -106.0],
                     [31.16, -105.73],
                     [30.92, -105.45],
                     [30.75, -105.11],
                     [30.67, -104.95],
                     [30.39, -104.85],
                     [30.19, -104.69],
                     [30.01, -104.69],
                     [29.79, -104.59],
                     [29.63, -104.51],
                     [29.53, -104.37],
                     [29.49, -104.2],
                     [29.39, -104.14],
                     [29.33

In [259]:
# convert the dataframe to a dictionary then to a JSON string
airsigmet_string=json.dumps(list(airsigmet_data_df.T.to_dict().values()))

# open the file in write mode
# output_path = os.path.join("Resources", "airsigmet_data.json")
output_path = os.path.join("", "airsigmet_data.json")
with open(output_path, "w") as file:
    # write the JSON string to the file
    file.write(airsigmet_string.replace("NaN","null"))

# file is automatically closed after the with block